In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE104958"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE104958"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE104958.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE104958.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE104958.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A 17-molecule set as a predictor of complete response to neoadjuvant chemotherapy with docetaxel, cisplatin, and 5-fluorouracil in esophageal cancer"
!Series_summary	"Background  Recently, neoadjuvant chemotherapy with docetaxel/cisplatin/5-fluorouracil (NAC-DCF) was identified as a novel strong regimen with a high rate of pathological complete response (pCR) in advanced esophageal cancer in Japan. Predicting pCR will contribute to the therapeutic strategy and the prevention of surgical invasion. However, a predictor of pCR after NAC-DCF has not yet been developed. The aim of this study was to identify a novel predictor of pCR in locally advanced esophageal cancer treated with NAC-DCF.  Patients and Methods   A total of 32 patients who received NAC-DCF followed by esophagectomy between June 2013 and March 2016 were enrolled in this study. We divided the patients into the following 2 groups: pCR group (9 cases) and non-pCR group (23 cases), and com

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
import json
from typing import Dict, Any, Optional, Callable, List
import numpy as np

# 1. Check if gene expression data is available
# Based on the background information, this dataset contains DNA microarray data
# The study specifically mentions gene expression analysis in esophageal cancer tissues
is_gene_available = True

# 2. Variable availability and data type conversion

# 2.1 Data Availability
# From the sample characteristics dictionary, we can see:
# Key 1 has information about tissue type (cancer vs normal tissue)
# For tissue: we can identify cancer/normal samples which is related to the trait
trait_row = 1  # The row containing cancer/normal tissue information

# There is no explicit age information in the sample characteristics
age_row = None

# There is no explicit gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """
    Convert tissue information to binary trait values.
    0 = normal tissue, 1 = cancer tissue
    """
    if not value or pd.isna(value):
        return None
    
    # Get the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'cancer' in value.lower():
        return 1
    elif 'normal' in value.lower():
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age information to float.
    This function is defined but not used since age data is not available.
    """
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender information to binary.
    This function is defined but not used since gender data is not available.
    """
    return None

# 3. Save Metadata
# We need to check if trait data is available
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # Load the clinical data - create a properly structured DataFrame from the sample characteristics
    # Create a dictionary where all lists have the same length
    clinical_data_dict = {
        0: ['organ: esophagus', 'organ: esophagus'],  # Duplicate to match length
        1: ['tissue: cancer tissue', 'tissue: normal tissue']
    }
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame.from_dict(clinical_data_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [1.0], 1: [0.0]}
Clinical data saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE104958.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 68
Header line: "ID_REF"	"GSM2811122"	"GSM2811123"	"GSM2811124"	"GSM2811125"	"GSM2811126"	"GSM2811127"	"GSM2811128"	"GSM2811129"	"GSM2811130"	"GSM2811131"	"GSM2811132"	"GSM2811133"	"GSM2811134"	"GSM2811135"	"GSM2811136"	"GSM2811137"	"GSM2811138"	"GSM2811139"	"GSM2811140"	"GSM2811141"	"GSM2811142"	"GSM2811143"	"GSM2811144"	"GSM2811145"	"GSM2811146"	"GSM2811147"	"GSM2811148"	"GSM2811149"	"GSM2811150"	"GSM2811151"	"GSM2811152"	"GSM2811153"	"GSM2811154"	"GSM2811155"	"GSM2811156"	"GSM2811157"	"GSM2811158"	"GSM2811159"	"GSM2811160"	"GSM2811161"	"GSM2811162"	"GSM2811163"	"GSM2811164"	"GSM2811165"	"GSM2811166"	"GSM2811167"
First data line: "(+)E1A_r60_1"	0.12522125	0.100678444	-0.19952202	0.20687675	-0.6983681	0.011138916	-0.24957657	-0.25128937	0.31498814	-0.26011658	-0.8179703	0.21413231	-0.19338608	0.2784319	-0.84678745	0.28999233	-0.09785175	0.6191282	0.25938606	-0.13983536	0.78502464	-0.18253994	0.53778267	0.53588676	0.38568115	-0.37277508	0.76341057	-0.22571278	

Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the provided gene identifiers:
# The IDs include items like "(+)E1A_r60_1", "3xSLv1", and "A_19_P00315452"
# These do not appear to be standard human gene symbols (like BRCA1, TP53, etc.)
# The "A_19_P" prefix suggests these are likely probe IDs from an Agilent microarray platform

# These identifiers will need to be mapped to standard gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE104958
Line 6: !Series_title = A 17-molecule set as a predictor of complete response to neoadjuvant chemotherapy with docetaxel, cisplatin, and 5-fluorouracil in esophageal cancer
Line 7: !Series_geo_accession = GSE104958
Line 8: !Series_status = Public on Oct 14 2017
Line 9: !Series_submission_date = Oct 13 2017
Line 10: !Series_last_update_date = Jul 25 2021
Line 11: !Series_pubmed_id = 29136005
Line 12: !Series_summary = Background  Recently, neoadjuvant chemotherapy with docetaxel/cisplatin/5-fluorouracil (NAC-DCF) was identified as a novel strong regimen with a high rate of pathological complete response (pCR) in advanced esophageal cancer in Japan. Predicting pCR will contribute to the therapeutic 


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_00110553

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the preview of gene annotation data, let's identify the relevant columns:
# - 'ID' or 'SPOT_ID' contains the probe identifiers (like A_33_P3396872)
# - 'GENE_SYMBOL' contains the gene symbols (like CPED1, BCOR)

# 1. Decide which columns to use for mapping
probe_col = 'ID'  # The column containing probe IDs that match gene_data index
gene_symbol_col = 'GENE_SYMBOL'  # The column containing gene symbols

# 2. Get a gene mapping dataframe
# First, ensure we have the gene_annotation dataframe properly loaded
# If the previous attempt worked, we should already have gene_annotation

# Let's try to use the get_gene_annotation function again with proper error handling
try:
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation with {len(gene_annotation)} rows")
except Exception as e:
    print(f"Error with get_gene_annotation: {e}")
    # Use our previously created dataframe from the platform section
    # We assume it's already in the gene_annotation variable

# Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_symbol_col)
print(f"Created mapping dataframe with {len(mapping_df)} rows")
print(f"Sample of mapping dataframe:\n{mapping_df.head()}")

# 3. Apply gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Successfully mapped probe data to gene-level data with {len(gene_data)} genes")
print(f"First few gene symbols in gene_data: {gene_data.index[:5]}")

# Normalize gene symbols (optional but recommended)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalizing gene symbols, gene_data has {len(gene_data)} genes")
print(f"First few normalized gene symbols: {gene_data.index[:5]}")

# Saving the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Successfully extracted gene annotation with 2742073 rows
Created mapping dataframe with 48862 rows
Sample of mapping dataframe:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146


Successfully mapped probe data to gene-level data with 29222 genes
First few gene symbols in gene_data: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3'], dtype='object', name='Gene')
After normalizing gene symbols, gene_data has 20778 genes
First few normalized gene symbols: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE104958.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20778, 46)
First few genes with their expression values after normalization:
          GSM2811122  GSM2811123  GSM2811124  GSM2811125  GSM2811126  \
Gene                                                                   
A1BG        0.103348   -0.582059   -1.208877    2.239748    0.195371   
A1BG-AS1   -0.048602   -0.208037   -1.792347    1.689534    0.342461   
A1CF       -0.105821   -0.245147   -0.681063   -0.207984   -1.054633   
A2M        -0.015755   -0.460151    0.004467   -0.690533    1.742541   
A2M-AS1     0.220449   -0.347176   -0.256770   -2.044582    0.581432   

          GSM2811127  GSM2811128  GSM2811129  GSM2811130  GSM2811131  ...  \
Gene                                                                  ...   
A1BG       -0.619633    1.618708    0.673897    0.165190   -0.173507  ...   
A1BG-AS1    0.003299    1.232518    0.849790   -0.003299    0.348321  ...   
A1CF       -0.714191   -1.024918   -0.998374   -0.243343   -1.165812  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Esophageal_Cancer/gene_data/GSE104958.csv
Raw clinical data shape: (2, 47)
Clinical features:
                   GSM2811122  GSM2811123  GSM2811124  GSM2811125  GSM2811126  \
Esophageal_Cancer         1.0         1.0         1.0         1.0         1.0   

                   GSM2811127  GSM2811128  GSM2811129  GSM2811130  GSM2811131  \
Esophageal_Cancer         1.0         1.0         1.0         1.0         1.0   

                   ...  GSM2811158  GSM2811159  GSM2811160  GSM2811161  \
Esophageal_Cancer  ...         1.0         0.0         1.0         1.0   

                   GSM2811162  GSM2811163  GSM2811164  GSM2811165  GSM2811166  \
Esophageal_Cancer         1.0         1.0         1.0         1.0         1.0   

                   GSM2811167  
Esophageal_Cancer         1.0  

[1 rows x 46 columns]
Clinical features saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE104958.csv
Linked data shape: (46, 20779)

Data shape after handling missing values: (46, 20779)
For the feature 'Esophageal_Cancer', the least common label is '0.0' with 5 occurrences. This represents 10.87% of the dataset.
The distribution of the feature 'Esophageal_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Esophageal_Cancer/GSE104958.csv
